In [4]:
!pip install -q pyomo
!apt install -y -q coinor-cbc

Reading package lists...
Building dependency tree...
Reading state information...
coinor-cbc is already the newest version (2.10.3+repack1-1build1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [5]:
import numpy as np
import pandas as pd
from pyomo.environ import *

In [6]:
distances = pd.read_excel("/content/distance_matirx.xlsx", index_col=0)
distances.head()

,Sangli,Satara,Kolhapur,Pune,Akola,Amravati,Yavatmal,Aurangabad,Jalna,Osmanabad,...,Nagpur,Wardha,Nandurbar,Nashik,Ahmednagar,Palghar,Raigad,Ratnagiri,Sindhudurg,Mumbai
Sangli,0,122,47,230,683,751,779,499,475,257,...,890,818,659,449,294,470,265,172,178,373
Satara,122,0,122,110,575,643,671,306,369,307,...,782,710,538,328,193,350,142,192,254,253
Kolhapur,47,122,0,230,726,794,822,426,518,299,...,933,861,659,449,313,470,264,132,129,373
Pune,230,110,230,0,505,573,601,236,300,316,...,712,641,423,213,123,248,139,303,368,151
Akola,683,575,726,505,0,98,166,250,197,428,...,277,205,344,421,386,568,655,769,857,575


In [7]:
def TSPModel(distance):
    Len = len(distance)
    mod = ConcreteModel()

    mod.x = Var(range(Len), range(Len), domain=Binary)
    mod.u = Var(range(Len), domain=PositiveIntegers)
        
    mod.constraints = ConstraintList()
    for i in range(Len):
        mod.constraints.add(sum(mod.x[i, l] for l in range(Len) if i != l) == 1)
        mod.constraints.add(sum(mod.x[l, i] for l in range(Len) if i != l) == 1)

    mod.constraints.add(mod.u[0] == 1)
    for i in range(1, Len):
        mod.u[i].setlb(2)
        mod.u[i].setub(Len)
        if i!=0 :
            for j in range(Len):
                if j != 0:
                    mod.constraints.add(mod.u[i] - mod.u[j] + 1 <= (Len-1)*(1-mod.x[i, j]))
        
    # Model objective funciton defined
    mod.objective = Objective(expr=sum(distance[i, j]*mod.x[i, j] for i in range(Len) for j in range(Len)))
    
    return mod

In [8]:
distance_cities = distances.to_numpy()
TSPmodel = TSPModel(distance_cities)

In [9]:
integer_solver = SolverFactory("cbc", executable='/usr/bin/cbc')

In [10]:
Answer =integer_solver.solve(TSPmodel)
print(Answer)
print("Status of TSP solver : ", Answer.solver.status)
print("Termination Condition of TSP solver: ", Answer.solver.termination_condition)


Problem: 
- Name: unknown
  Lower bound: 3600.0
  Upper bound: 3600.0
  Number of objectives: 1
  Number of constraints: 602
  Number of variables: 624
  Number of binary variables: 624
  Number of integer variables: 649
  Number of nonzeros: 600
  Sense: minimize
Solver: 
- Status: ok
  User time: -1.0
  System time: 22.24
  Wallclock time: 24.34
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1210
      Number of created subproblems: 1210
    Black box: 
      Number of iterations: 71571
  Error rc: 0
  Time: 24.412513494491577
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Status of TSP solver :  ok
Termination Condition of TSP solver:  optimal


In [11]:
print("Minimized Objective value : ", TSPmodel.objective())

Minimized Objective value :  3600.0


By minimising the distance travelled while taking into account all the constraints, we were able to use the Travelling Salesman Problem model to resolve the issue.
Time taken : 21 secs

Solver used : cbc